In [1]:
import akshare as ak
import pandas as pd
import numpy as np

def fetch_and_merge_convertible_bonds():
    """
    获取东方财富和同花顺的历史可转债数据，
    进行比对和合并，并将最终结果保存到Excel文件中。
    """
    print("🚀 开始执行任务：获取、比对并合并历史可转债数据...")

    # --- 步骤 1: 从不同数据源获取可转债列表 ---
    print("\n[步骤 1/4] 正在从东方财富获取可转债列表...")
    try:
        # 东方财富接口，数据相对全面，作为我们的基础列表
        em_bonds_df = ak.bond_zh_cov()
        print(f"  ✅ 从东方财富成功获取 {len(em_bonds_df)} 条可转债记录。")
    except Exception as e:
        print(f"  ❌ 从东方财富获取数据失败，原因: {e}")
        return

    print("\n[步骤 1/4] 正在从同花顺获取可转债列表...")
    try:
        # 同花顺接口，作为交叉验证和补充的数据源
        ths_bonds_df = ak.bond_zh_cov_info_ths()
        print(f"  ✅ 从同花顺成功获取 {len(ths_bonds_df)} 条可转债记录。")
    except Exception as e:
        print(f"  ❌ 从同花顺获取数据失败，原因: {e}")
        return

    # --- 步骤 2: 数据合并与差异识别 ---
    print("\n[步骤 2/4] 正在合并两个数据源并识别差异...")
    
    # 使用 'outer' 方式合并，确保两个列表中的所有记录都被保留
    # 使用 'indicator=True' 参数来自动标记每条记录的来源
    # 使用 suffixes 来处理两个数据源中重名的列
    merged_df = pd.merge(
        em_bonds_df,
        ths_bonds_df,
        on='债券代码',
        how='outer',
        suffixes=('_em', '_ths'),
        indicator=True
    )

    # 对来源标识列进行重命名和转换，使其更具可读性
    # 'both': 两边都有, 'left_only': 仅东方财富有, 'right_only': 仅同花顺有
    source_map = {
        'both': '共同存在',
        'left_only': '仅东方财富',
        'right_only': '仅同花顺'
    }
    merged_df['_source_check'] = merged_df['_merge'].map(source_map)
    merged_df.drop('_merge', axis=1, inplace=True)
    
    # 统计并打印差异信息
    source_counts = merged_df['_source_check'].value_counts()
    print("  ✅ 数据合并完成，来源分布如下:")
    print(f"    - 两个平台共同拥有的债券数量: {source_counts.get('共同存在', 0)}")
    print(f"    - 仅在东方财富中存在的债券数量: {source_counts.get('仅东方财富', 0)}")
    print(f"    - 仅在同花顺中存在的债券数量: {source_counts.get('仅同花顺', 0)}")


    # --- 步骤 3: 整理与优化DataFrame ---
    print("\n[步骤 3/4] 正在整理列顺序以便于查看...")
    
    # 将关键信息和标识列提前，方便快速浏览
    # 首先定义我们希望排在最前面的列
    preferred_columns = [
        '债券代码',
        '_source_check',
        '债券简称_em',
        '债券简称_ths',
        '正股代码_em',
        '正股简称_em',
        '正股代码_ths',
        '正股简称_ths',
        '上市时间', # 这个来自东方财富，同花顺的列名是'上市日期'
        '上市日期',   # 这个来自同花顺
    ]
    
    # 获取所有其他列，并从其他列中移除我们已经指定的优先列
    other_columns = [col for col in merged_df.columns if col not in preferred_columns]
    
    # 拼接列顺序
    final_column_order = preferred_columns + sorted(other_columns) # 对剩余列进行排序
    
    # 应用新的列顺序
    final_df = merged_df[final_column_order]
    print("  ✅ 列顺序调整完毕。")

    # --- 步骤 4: 导出到Excel ---
    output_filename = "历史可转债数据全集.xlsx"
    print(f"\n[步骤 4/4] 正在将结果导出到Excel文件: {output_filename} ...")
    try:
        final_df.to_excel(output_filename, index=False)
        print(f"  🎉 任务成功完成！数据已保存至您的当前目录下的 '{output_filename}' 文件中。")
    except Exception as e:
        print(f"  ❌ 导出Excel失败，原因: {e}")

if __name__ == '__main__':
    # 执行主函数
    fetch_and_merge_convertible_bonds()

🚀 开始执行任务：获取、比对并合并历史可转债数据...

[步骤 1/4] 正在从东方财富获取可转债列表...


  0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\zhuiz\AppData\Roaming\Python\Python313\site-packages\akshare\bond\bond_zh_cov.py:342: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat(objs=[big_df, temp_df], ignore_index=True)


  ✅ 从东方财富成功获取 982 条可转债记录。

[步骤 1/4] 正在从同花顺获取可转债列表...
  ✅ 从同花顺成功获取 889 条可转债记录。

[步骤 2/4] 正在合并两个数据源并识别差异...
  ✅ 数据合并完成，来源分布如下:
    - 两个平台共同拥有的债券数量: 888
    - 仅在东方财富中存在的债券数量: 94
    - 仅在同花顺中存在的债券数量: 1

[步骤 3/4] 正在整理列顺序以便于查看...
  ✅ 列顺序调整完毕。

[步骤 4/4] 正在将结果导出到Excel文件: 历史可转债数据全集.xlsx ...
  🎉 任务成功完成！数据已保存至您的当前目录下的 '历史可转债数据全集.xlsx' 文件中。


In [11]:
import akshare as ak
import pandas as pd
from IPython.display import display, HTML

# --- 核心：中英文列名翻译字典 ---
# 我已经根据接口返回的字段，为你手动翻译了绝大部分常用字段
COLUMN_TRANSLATION_MAP = {
    # 通用
    'SECUCODE': '完整代码',
    'SECURITY_CODE': '债券代码',
    'SECURITY_NAME_ABBR': '债券简称',
    'TRADE_MARKET': '交易市场',
    'CONVERT_STOCK_CODE': '正股代码',
    'CORRECODE_NAME_ABBR': '正股简称',
    
    # 基本信息
    'LISTING_DATE': '上市日期',
    'BOND_EXPIRE': '债券年限(年)',
    'RATING': '信用评级',
    'ISSUE_YEAR': '发行年份',
    'CEASE_DATE': '摘牌日',
    'EXPIRE_DATE': '到期日期',
    'INTEREST_RATE_EXPLAIN': '利率说明',
    'ACTUAL_ISSUE_SCALE': '实际发行规模(亿元)',
    'ISSUE_PRICE': '发行价格(元)',
    'PAR_VALUE': '票面价值(元)',
    'ISSUE_OBJECT': '发行对象',
    'CONVERT_STOCK_PRICE': '正股价',
    'TRANSFER_PRICE': '转股价(元)',
    'TRANSFER_VALUE': '转股价值(元)',
    'CURRENT_BOND_PRICE': '当前债价(元)',
    'TRANSFER_PREMIUM_RATIO': '转股溢价率(%)',
    'RESALE_CLAUSE': '回售条款',
    'REDEEM_CLAUSE': '赎回条款',
    'RESALE_TRIG_PRICE': '回售触发价(元)',
    'REDEEM_TRIG_PRICE': '强赎触发价(元)',
    'PBV_RATIO': '市净率PB',
    'IS_CONVERT_STOCK': '是否可转股',
    'IS_REDEEM': '是否可赎回',
    'IS_SELLBACK': '是否可回售',
    
    # 中签号
    'TYPE': '中签号类型',
    'BALLOT_NUM': '中签号码',
    
    # 筹资用途
    'ITEM_NAME': '项目名称',
    'PLAN_INVEST_AMT': '计划投资金额(万元)',
    'PLAN_INPUT_RF': '计划投入募集资金(万元)',
    'YIELD': '项目收益率(%)',
    'INVEST_RECOVERY_PERIOD': '投资回收期(月)',
    'ITEM_EXPLAIN': '项目概况',
    'ITEM_PROCESS_EXPLAIN': '项目进度',

    # 重要日期
    'SECURITY_NAME': '证券名称',
    'EVENT_TYPE_CODE_NAME': '事件类型',
    'EVENT_CONTENT': '事件内容',
    'EVENT_DATE': '日期'
}


def get_and_display_convertible_bond_details(bond_code: str):
    """
    获取并以美观的表格形式展示指定可转债的详细信息。
    代码已适配新版akshare接口，并包含中文表头翻译。

    参数:
    bond_code (str): 6位的纯数字可转债代码，例如 "111023"。
    """
    print(f"🚀 开始获取可转债 {bond_code} 的详细档案...")

    indicators = ["基本信息", "重要日期", "中签号", "筹资用途"]

    for indicator in indicators:
        # 使用HTML H3标签来创建更醒目的标题
        display(HTML(f"<h3>查询模块：{indicator}</h3>"))
        
        try:
            df = ak.bond_zh_cov_info(symbol=bond_code, indicator=indicator)

            if df.empty:
                print("  - 未查询到相关数据。")
                continue
            
            # --- 核心逻辑调整 ---
            # 1. 重命名列为中文
            df_renamed = df.rename(columns=COLUMN_TRANSLATION_MAP)

            # 2. 根据数据结构选择最佳展示方式
            # 对于只有一行宽表的“基本信息”和“重要日期”，转置后更易读
            if indicator in ["基本信息", "重要日期"] and len(df_renamed) == 1:
                # 删除一些不必要的原始ID列，让表格更干净
                columns_to_drop = [col for col in df_renamed.columns if col not in COLUMN_TRANSLATION_MAP.values()]
                df_cleaned = df_renamed.drop(columns=columns_to_drop, errors='ignore')
                
                # 转置DataFrame
                display_df = df_cleaned.T.reset_index()
                display_df.columns = ['项目', '内容']
                display(display_df)
            else:
                # 对于多行的“中签号”和“筹资用途”，直接展示表格
                display(df_renamed)
        
        except Exception as e:
            print(f"  ❌ 查询 '{indicator}' 时发生错误: {e}")
            print("  - 请检查输入的债券代码是否正确，或接口是否支持该数据。")

    print(f"\n🎉 任务完成！已展示可转债 {bond_code} 的全部可查询档案。")


if __name__ == '__main__':
    # --- 使用示例 ---
    # 请将这里的代码粘贴到一个Jupyter Notebook或IPython环境中运行，以获得最佳的表格渲染效果。
    # 替换为你想要查询的可转债代码，这里以利柏转债 "111023" 为例
    
    target_bond_code = "111023"
    get_and_display_convertible_bond_details(target_bond_code)


🚀 开始获取可转债 111023 的详细档案...


,项目,内容
0,债券代码,111023
1,完整代码,111023.SH
2,交易市场,CNSESH
3,债券简称,利柏转债
4,上市日期,2025-07-22 00:00:00
5,正股代码,605167
6,债券年限(年),6
7,信用评级,AA
8,发行年份,2025
9,摘牌日,2031-07-02 00:00:00


,BOND_COMBINE_CODE,债券代码,交易市场,债券简称,完整代码,DATE_TYPE_CODE,DATE_TYPE,IS_INTERVAL,START_DATE,END_DATE,NUM
0,25270600001PKI,111023,CNSESH,利柏转债,111023.SH,0453,债券承销期,1,2025-07-01 00:00:00,2025-07-09 00:00:00,1
1,25270600001PKI,111023,CNSESH,利柏转债,111023.SH,0405,原股东优先配售股权登记日,0,2025-07-02 00:00:00,None,2
2,25270600001PKI,111023,CNSESH,利柏转债,111023.SH,0402,网上路演推荐日,0,2025-07-02 00:00:00,None,3
3,25270600001PKI,111023,CNSESH,利柏转债,111023.SH,0448,网上申购配号日,0,2025-07-03 00:00:00,None,4
4,25270600001PKI,111023,CNSESH,利柏转债,111023.SH,0425,网上中签率确定日,0,2025-07-03 00:00:00,None,5
5,25270600001PKI,111023,CNSESH,利柏转债,111023.SH,0419,网上摇号抽签日,0,2025-07-04 00:00:00,None,6
6,25270600001PKI,111023,CNSESH,利柏转债,111023.SH,0475,网上中签结果公告日,0,2025-07-07 00:00:00,None,7
7,25270600001PKI,111023,CNSESH,利柏转债,111023.SH,0451,债券资金到账日,0,2025-07-09 00:00:00,None,8
8,25270600001PKI,111023,CNSESH,利柏转债,111023.SH,0436,上交所上市日期披露时间,0,2025-07-18 00:00:00,None,9


,完整代码,债券代码,BOND_COMBINE_CODE,债券简称,TRADE_MARKET_CODE,TYPE_CODE,中签号类型,中签号码
0,111023.SH,111023,25270600001PKI,利柏转债,069001001,5,"末""五""位数",53998
1,111023.SH,111023,25270600001PKI,利柏转债,069001001,6,"末""六""位数","083273,283273,399760,483273,683273,883273,899760"
2,111023.SH,111023,25270600001PKI,利柏转债,069001001,7,"末""七""位数","2013918,4605793,9605793"
3,111023.SH,111023,25270600001PKI,利柏转债,069001001,8,"末""八""位数","10706377,35706377,60706377,85706377"
4,111023.SH,111023,25270600001PKI,利柏转债,069001001,9,"末""九""位数","050732373,250732373,426046050,450732373,650732..."
5,111023.SH,111023,25270600001PKI,利柏转债,069001001,11,"末""十""位数","1904001711,1983256906,2392319135,4531502457,60..."


,完整代码,债券代码,BOND_NAME_ABBR,BOND_COMBINE_CODE,ITEM_NUM,项目名称,计划投资金额(万元),计划投入募集资金(万元),项目收益率(%),投资回收期(月),CURRENCY,ACTUAL_INPUT_RF,PRETAX_INTERNAL_YIELD,PRETAX_INVEST_RECOVERYPRD,交易市场,项目概况,项目进度,BUILD_PERIOD,SORT,INVEST_RETURN_1Y
0,111023.SH,111023,利柏转债,25270600001PKI,1,南通利柏特重工有限公司大型工业模块制造项目,129481.04,75000,13.05,72.72,CNY,49608.35,None,None,CNSESH,"本次发行的可转债所募集资金总额为75,000.00万元(含本数),扣除发行费用后,用于南通利...","2025.08.02公告\r\n截至2025年7月9日,司拟使用募集资金置换预先投入募投项目...",None,1,6.06



🎉 任务完成！已展示可转债 111023 的全部可查询档案。


In [10]:
import akshare as ak
import pandas as pd
import time

def get_single_bond_adjustment_history(bond_code: str):
    """
    获取指定可转债的转股价调整历史记录。

    Args:
        bond_code (str): 需要查询的可转债代码，例如 "128013"。
    """
    print(f"🚀 开始查询可转债 {bond_code} 的转股价调整历史...")

    # --- 步骤 1: 获取债券简称，让输出更友好 ---
    bond_name = "未知简称" # 默认值
    try:
        # 借用可转债列表接口快速查询简称
        bond_list_df = ak.bond_zh_cov()
        name_series = bond_list_df[bond_list_df['债券代码'] == bond_code]['债券简称']
        if not name_series.empty:
            bond_name = name_series.iloc[0]
        print(f"  - 查找到债券简称为: {bond_name}")
    except Exception as e:
        print(f"  ⚠️ 查询债券简称时遇到问题: {e}，将使用默认名称。")

    # --- 步骤 2: 查询转股价调整记录 ---
    try:
        print(f"  - 正在从集思录获取 {bond_code} 的调整记录...")
        adj_log_df = ak.bond_cb_adj_logs_jsl(symbol=bond_code)

        # --- 步骤 3: 处理与输出 ---
        if adj_log_df.empty:
            print(f"\n✅ 查询完成：未找到可转债 {bond_code} ({bond_name}) 的任何转股价调整记录。")
            return

        print("  - 数据获取成功，正在整理格式...")
        # 为表格添加代码和简称列，便于识别
        adj_log_df['债券代码'] = bond_code
        adj_log_df['债券简称'] = bond_name
        
        # 调整列顺序，让关键信息更靠前
        cols_order = [
            '债券代码', '债券简称', '股东大会日', '下修前转股价', 
            '下修后转股价', '新转股价生效日期', '下修底价'
        ]
        final_df = adj_log_df[cols_order].sort_values(by='新转股价生效日期').reset_index(drop=True)
        
        # 在Jupyter中直接美观地展示DataFrame
        print("\n--- 查询结果 ---")
        display(final_df) # 在Jupyter中，display()比print()更适合展示DataFrame

        # 导出到Excel
        output_filename = f"{bond_code}_{bond_name}_转股价调整记录.xlsx"
        final_df.to_excel(output_filename, index=False)
        print(f"\n🎉 任务成功完成！数据已保存至您的当前目录下的 '{output_filename}' 文件中。")

    except Exception as e:
        print(f"\n❌ 查询或处理过程中发生错误: {e}")
        print("   请检查输入的债券代码是否正确，或网络连接是否正常。")

# --- 如何使用 ---
if __name__ == '__main__':
    # 请在这里输入你想要查询的可转债代码
    bond_code_to_query = "128013"  # 这是一个很好的例子，因为它有多条调整记录
    
    # 调用函数执行查询
    get_single_bond_adjustment_history(bond_code_to_query)
    
    # 你也可以查询一个没有调整记录的例子
    # print("\n" + "="*50 + "\n")
    # get_single_bond_adjustment_history(bond_code="110088")

🚀 开始查询可转债 128013 的转股价调整历史...


  0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\zhuiz\AppData\Roaming\Python\Python313\site-packages\akshare\bond\bond_zh_cov.py:342: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat(objs=[big_df, temp_df], ignore_index=True)


  - 查找到债券简称为: 洪涛转债
  - 正在从集思录获取 128013 的调整记录...
  - 数据获取成功，正在整理格式...

--- 查询结果 ---


,债券代码,债券简称,股东大会日,下修前转股价,下修后转股价,新转股价生效日期,下修底价
0,128013,洪涛转债,2019-11-20,9.97,8.00,2019-11-21,3.10
1,128013,洪涛转债,2020-06-29,8.00,3.12,2020-06-30,3.12
2,128013,洪涛转债,2021-02-23,3.10,2.32,2021-02-24,2.32



🎉 任务成功完成！数据已保存至您的当前目录下的 '128013_洪涛转债_转股价调整记录.xlsx' 文件中。
